## Total Aggregations

Let us go through the details related to total aggregations using Spark.

* We can perform total aggregations directly on Dataframe or we can perform aggregations after grouping by a key(s).
* Here are the functions which we typically use to perform aggregations.
  * count
  * sum, avg
  * min, max

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [ ]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Basic Transformations'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

In [ ]:
airtraffic_path = "/public/airtraffic_all/airtraffic-part/flightmonth=200801"

In [ ]:
airtraffic = spark. \
    read. \
    parquet(airtraffic_path)

In [ ]:
from pyspark.sql.functions import concat, lpad, count, lit

* Get number of flights in the month of 2008 January.

In [ ]:
airtraffic.count()

In [ ]:
airtraffic.select(count(lit(1)).alias('count')).show()

In [ ]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth'). \
    describe(). \
    show()

In [ ]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth'). \
    summary(). \
    show()

* Get number of distinct dates from airtraffic data frame which is created using 2008 January data.

In [ ]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth'). \
    distinct(). \
    count()

In [ ]:
from pyspark.sql.functions import countDistinct

In [ ]:
airtraffic. \
    select(countDistinct('Year', 'Month', 'DayOfMonth').alias('countDistinct')). \
    show()

* Get the total bonus amount from employees data set. We need to use `sum` to get total bonus amount. We also have functions such as `min`, `max`, `avg` etc to take care of common aggregations.

In [ ]:
employees = [(1, "Scott", "Tiger", 1000.0, 10,
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, None,
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, '',
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 10,
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]

In [ ]:
employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, bonus STRING, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )

In [ ]:
employeesDF.show()

In [ ]:
from pyspark.sql.functions import col, coalesce, sum

In [ ]:
employeesDF. \
    select(((sum(coalesce(col('bonus').cast('int'), lit(0)) * col('salary'))) / lit(100)).alias('total_bonus')). \
    show()

In [ ]:
employeesDF. \
    selectExpr('sum((coalesce(cast(bonus AS INT), 0) * salary) / 100) AS total_bonus'). \
    show()

* Get revenue generated for a given order from order_items.

In [ ]:
order_items = spark.read.json('/public/retail_db_json/order_items')

In [ ]:
order_id = input('Enter order_id:')

In [ ]:
order_items. \
    filter(f'order_item_order_id = {int(order_id)}'). \
    select(sum('order_item_subtotal').alias('order_revenue')). \
    show()